In [2]:
import argparse
import copy
import time
from collections import deque

import cv2 as cv
import numpy as np
import mediapipe as mp
import tensorflow as tf


import argparse
import copy
import time
from collections import deque

import cv2 as cv
import numpy as np
import mediapipe as mp
import tensorflow as tf

# ----------------------------------
# Загрузка модели ДОЛЖНА быть здесь, ДО main()
# ----------------------------------
model = tf.keras.models.load_model("asl_model.h5")
class_names = [
    'A','B','C','D','E','F','G','H','I','J','K','L','M',
    'N','O','P','Q','R','S','T','U','V','W','X','Y','Z',
    'del','nothing','space'
]

class CvFpsCalc:
    def __init__(self, buffer_len=10):
        self._times = deque(maxlen=buffer_len)

    def get(self):
        now = time.time()
        self._times.append(now)
        if len(self._times) >= 2:
            period = self._times[-1] - self._times[0]
            fps = (len(self._times) - 1) / period if period > 0 else 0.0
        else:
            fps = 0.0
        return fps


def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--device", type=int, default=0)
    parser.add_argument("--width", type=int, default=960)
    parser.add_argument("--height", type=int, default=540)
    parser.add_argument("--min_detection_confidence", type=float, default=0.7)
    parser.add_argument("--min_tracking_confidence", type=float, default=0.5)
    args, _ = parser.parse_known_args()
    return args

def extract_and_preprocess_roi(image_bgr, brect):
    x1, y1, x2, y2 = brect
    h, w, _ = image_bgr.shape
    x1 = max(x1, 0)
    y1 = max(y1, 0)
    x2 = min(x2, w)
    y2 = min(y2, h)
    if x2 - x1 <= 0 or y2 - y1 <= 0:
        return None
    roi = image_bgr[y1:y2, x1:x2]
    roi_gray = cv.cvtColor(roi, cv.COLOR_BGR2GRAY)
    roi_resized = cv.resize(roi_gray, (60, 60))
    roi_norm = roi_resized.astype("float32") / 255.0
    img_tensor = np.expand_dims(np.expand_dims(roi_norm, axis=-1), axis=0)
    return img_tensor


def draw_landmarks(image, landmark_point):
    if len(landmark_point) > 0:
        # Thumb
        cv.line(image, tuple(landmark_point[2]), tuple(landmark_point[3]), (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[2]), tuple(landmark_point[3]), (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[3]), tuple(landmark_point[4]), (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[3]), tuple(landmark_point[4]), (255, 255, 255), 2)
        # Index finger
        cv.line(image, tuple(landmark_point[5]), tuple(landmark_point[6]), (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[5]), tuple(landmark_point[6]), (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[6]), tuple(landmark_point[7]), (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[6]), tuple(landmark_point[7]), (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[7]), tuple(landmark_point[8]), (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[7]), tuple(landmark_point[8]), (255, 255, 255), 2)
        # Middle finger
        cv.line(image, tuple(landmark_point[9]), tuple(landmark_point[10]), (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[9]), tuple(landmark_point[10]), (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[10]), tuple(landmark_point[11]), (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[10]), tuple(landmark_point[11]), (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[11]), tuple(landmark_point[12]), (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[11]), tuple(landmark_point[12]), (255, 255, 255), 2)
        # Ring finger
        cv.line(image, tuple(landmark_point[13]), tuple(landmark_point[14]), (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[13]), tuple(landmark_point[14]), (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[14]), tuple(landmark_point[15]), (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[14]), tuple(landmark_point[15]), (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[15]), tuple(landmark_point[16]), (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[15]), tuple(landmark_point[16]), (255, 255, 255), 2)
        # Little finger
        cv.line(image, tuple(landmark_point[17]), tuple(landmark_point[18]), (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[17]), tuple(landmark_point[18]), (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[18]), tuple(landmark_point[19]), (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[18]), tuple(landmark_point[19]), (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[19]), tuple(landmark_point[20]), (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[19]), tuple(landmark_point[20]), (255, 255, 255), 2)
        # Palm
        cv.line(image, tuple(landmark_point[0]), tuple(landmark_point[1]), (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[0]), tuple(landmark_point[1]), (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[1]), tuple(landmark_point[2]), (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[1]), tuple(landmark_point[2]), (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[2]), tuple(landmark_point[5]), (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[2]), tuple(landmark_point[5]), (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[5]), tuple(landmark_point[9]), (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[5]), tuple(landmark_point[9]), (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[9]), tuple(landmark_point[13]), (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[9]), tuple(landmark_point[13]), (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[13]), tuple(landmark_point[17]), (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[13]), tuple(landmark_point[17]), (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[17]), tuple(landmark_point[0]), (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[17]), tuple(landmark_point[0]), (255, 255, 255), 2)

    # Draw keypoints
    for index, landmark in enumerate(landmark_point):
        radius = 5
        if index in (4, 8, 12, 16, 20):
            radius = 8
        cv.circle(image, (landmark[0], landmark[1]), radius, (255, 255, 255), -1)
        cv.circle(image, (landmark[0], landmark[1]), radius, (0, 0, 0), 1)

    return image


def draw_bounding_rect(use_brect, image, brect):
    if use_brect:
        cv.rectangle(image, (brect[0], brect[1]), (brect[2], brect[3]), (0, 0, 0), 1)
    return image


def draw_info_text(image, brect, handedness, hand_sign_text, finger_gesture_text):
    cv.rectangle(image, (brect[0], brect[1]), (brect[2], brect[1] - 22), (0, 0, 0), -1)
    info_text = handedness.classification[0].label
    if hand_sign_text != "":
        info_text = info_text + ':' + hand_sign_text
    cv.putText(image, info_text, (brect[0] + 5, brect[1] - 4),
               cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv.LINE_AA)
    return image


def draw_point_history(image, point_history):
    for index, point in enumerate(point_history):
        if point[0] != 0 and point[1] != 0:
            cv.circle(image, (point[0], point[1]), 1 + int(index / 2), (152, 251, 152), 2)
    return image


def draw_info(image, fps, mode, number):
    cv.putText(image, "FPS:" + str(fps), (10, 30),
               cv.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 0), 4, cv.LINE_AA)
    cv.putText(image, "FPS:" + str(fps), (10, 30),
               cv.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2,
               cv.LINE_AA)
    return image


def calc_bounding_rect(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]
    landmark_array = np.empty((0, 2), int)
    for _, landmark in enumerate(landmarks.landmark):
        lx = min(int(landmark.x * image_width), image_width - 1)
        ly = min(int(landmark.y * image_height), image_height - 1)
        landmark_array = np.append(landmark_array, np.array([[lx, ly]]), axis=0)
    x, y, w, h = cv.boundingRect(landmark_array)
    return [x, y, x + w, y + h]


def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]
    landmark_point = []
    for _, landmark in enumerate(landmarks.landmark):
        lx = min(int(landmark.x * image_width), image_width - 1)
        ly = min(int(landmark.y * image_height), image_height - 1)
        landmark_point.append([lx, ly])
    return landmark_point


def main():
    args = get_args()
    cap_device = args.device
    cap_width = args.width
    cap_height = args.height
    min_det = args.min_detection_confidence
    min_track = args.min_tracking_confidence

    # Переменные для отслеживания «устойчивой» буквы
    prev_label = ""
    label_start_time = 0.0
    confirmed_text = ""
    added_flag = False

    # Новая переменная: время, когда в последний раз пользователь показывал любую букву ≠ "nothing"
    last_active_time = time.time()

    use_brect = True

    cap = cv.VideoCapture(cap_device)
    cap.set(cv.CAP_PROP_FRAME_WIDTH, cap_width)
    cap.set(cv.CAP_PROP_FRAME_HEIGHT, cap_height)

    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=1,
        min_detection_confidence=min_det,
        min_tracking_confidence=min_track,
    )

    cvFpsCalc = CvFpsCalc(buffer_len=10)
    point_history = deque(maxlen=16)

    while True:
        fps = cvFpsCalc.get()
        key = cv.waitKey(10)
        if key == 27:  # ESC
            break

        # Считываем кадр и зеркалим
        ret, image = cap.read()
        if not ret:
            break
        image = cv.flip(image, 1)

        # Создаём debug_image
        debug_image = copy.deepcopy(image)

        # Детекция
        image_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        image_rgb.flags.writeable = False
        results = hands.process(image_rgb)
        image_rgb.flags.writeable = True

        # Текущее время, чтобы проверять таймеры
        current_time = time.time()

        if results.multi_hand_landmarks is not None:
            for hand_landmarks, handedness in zip(results.multi_hand_landmarks,
                                                  results.multi_handedness):
                brect = calc_bounding_rect(image, hand_landmarks)
                landmark_list = calc_landmark_list(image, hand_landmarks)

                # Классификация ROI
                img_tensor = extract_and_preprocess_roi(image, brect)
                if img_tensor is not None:
                    preds = model.predict(img_tensor)
                    idx = np.argmax(preds[0])
                    predicted_label = class_names[idx]
                else:
                    predicted_label = ""

                point_history.append([0, 0])

                # Если жест ≠ "" и ≠ "nothing", обновляем last_active_time
                if predicted_label not in ("", "nothing"):
                    last_active_time = current_time

                # Логика подтверждения буквы ≥ 2 секунд
                if predicted_label == prev_label:
                    if (not added_flag
                            and predicted_label not in ("", "nothing")
                            and current_time - label_start_time >= 2.0):
                        confirmed_text += prev_label
                        added_flag = True
                else:
                    prev_label = predicted_label
                    label_start_time = current_time
                    added_flag = False

                # Рисуем bounding box, скелет и подпись жеста
                debug_image = draw_bounding_rect(use_brect, debug_image, brect)
                debug_image = draw_landmarks(debug_image, landmark_list)
                debug_image = draw_info_text(
                    debug_image,
                    brect,
                    handedness,
                    predicted_label,
                    ""
                )
        else:
            point_history.append([0, 0])

        # Если с момента last_active_time прошло ≥ 15 секунд, очищаем confirmed_text
        if current_time - last_active_time >= 10.0:
            confirmed_text = ""

        # Рисуем историю точек и FPS
        debug_image = draw_point_history(debug_image, point_history)
        debug_image = draw_info(debug_image, fps, 0, -1)  # FPS на (10,30)

        # Рисуем слово красным напротив FPS (справа, y=30)
        text = f"Word: {confirmed_text}"
        font = cv.FONT_HERSHEY_SIMPLEX
        font_scale = 2
        thickness = 2
        (text_width, text_height), baseline = cv.getTextSize(text, font, font_scale, thickness)
        frame_width = debug_image.shape[1]
        x = frame_width - text_width - 10
        y = 50  # та же высота, что и FPS

        cv.putText(
            debug_image,
            text,
            (x, y),
            font,
            font_scale,
            (0, 0, 255),  # красный цвет
            thickness,
            cv.LINE_AA
        )

        cv.imshow('Hand Gesture Recognition (Mirrored)', debug_image)

    cap.release()
    cv.destroyAllWindows()


In [3]:
if __name__ == '__main__':
    main()

I0000 00:00:1748963332.127790 1074179 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1748963332.162239 1088255 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748963332.172169 1088257 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


1/1 [==============================] - 0s 10ms/step


W0000 00:00:1748963337.417822 1088260 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
2025-06-03 18:08:57.450794: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 0s 11ms/step
